In [ ]:
# Code
import sys
sys.path.append('../')
from generative_formalism import *

In [ ]:
data = {}

In [ ]:
# sample
df_smpl = get_chadwyck_corpus_sampled_by('period')
df_smpl_rhyme = get_rhyme_for_sample(df_smpl, with_sample=True)
df_smpl_rhyme['pre_20C'] = ~df_smpl_rhyme.period.str.startswith('19')
df_smpl_rhyme.groupby('pre_20C').rhyme_pred_perc.mean()
pre20cdf = df_smpl_rhyme[df_smpl_rhyme.pre_20C]
rhyme_by_period_pre20c = pre20cdf.groupby('period').rhyme_pred_perc.mean()
rhyme_by_period = df_smpl_rhyme.groupby('period').rhyme_pred_perc.mean()
data['MIN_RHYME_PRED_PERC_PRE_20C'] = int(round(rhyme_by_period_pre20c.min()))
data['MAX_RHYME_PRED_PERC_PRE_20C'] = int(round(rhyme_by_period_pre20c.max()))
data['C19l_RHYME_PRED_PERC'] = int(round(rhyme_by_period['1850-1900']))
data['C20e_RHYME_PRED_PERC'] = int(round(rhyme_by_period['1900-1950']))
data['C20l_RHYME_PRED_PERC'] = int(round(rhyme_by_period['1950-2000']))
data

In [ ]:
# Number of prompted poems in total
df = get_genai_rhyme_promptings()
data['NUM_PROMPTED_POEMS'] = len(df)
df_rhyme = get_rhyme_for_sample(df, with_sample=True)
df_rhyme['model2'] = df_rhyme.model.apply(rename_model)
df_rhyme['prompt_type2'] = df_rhyme.prompt_type!='do_NOT_rhyme'
df_rhyme.groupby('prompt_type2').rhyme_pred_perc.mean()
data['AVG_RHYME_PRED_PERC_NOT_DO_NOT_RHYME'] = int(round(df_rhyme.groupby('prompt_type2').rhyme_pred_perc.mean()[True]))
data['AVG_RHYME_PRED_PERC_DO_NOT_RHYME'] = int(round(df_rhyme.groupby('prompt_type2').rhyme_pred_perc.mean()[False]))

maybe_rhyme = df_rhyme[df_rhyme.prompt_type=='MAYBE_rhyme']
data['MIN_RHYME_PRED_PERC_MAYBE_RHYME'] = int(round(maybe_rhyme.groupby('model2').rhyme_pred_perc.mean().min()))
data['MAX_RHYME_PRED_PERC_MAYBE_RHYME'] = int(round(maybe_rhyme.groupby('model2').rhyme_pred_perc.mean().max()))

no_rhyme = df_rhyme[df_rhyme.prompt_type=='do_NOT_rhyme']
no_rhyme_model = no_rhyme.groupby('model2').rhyme_pred_perc.mean()

data['NO_RHYME_GEMINI_PRED_PERC'] = int(round(no_rhyme_model['Gemini']))
data['NO_RHYME_CLAUDE_PRED_PERC'] = int(round(no_rhyme_model['Claude']))
data['NO_RHYME_CHATGPT_PRED_PERC'] = int(round(no_rhyme_model['ChatGPT']))
data['NO_RHYME_LLAMA_PRED_PERC'] = int(round(no_rhyme_model['Llama']))
data['NO_RHYME_DEEPSEEK_PRED_PERC'] = int(round(no_rhyme_model['DeepSeek']))
data['NO_RHYME_OLMO_PRED_PERC'] = int(round(no_rhyme_model['Olmo']))


data

In [ ]:
maybe_rhyme

In [ ]:
df.prompt_type.value_counts()

In [ ]:
# Number of completed poems
df = get_genai_rhyme_completions(by_line=False)
df['model2'] = df.model.map(rename_model)
df=df[df.model2!='']
len(df), df.id_human.nunique()

data['NUM_COMPLETED_POEMS'] = len(df)
data['NUM_COMPLETED_POEMS_UNIQUE'] = df.id_human.nunique()
ps=df.groupby('period').id_human.nunique().sort_values()
data['MIN_NUM_COMPLETIONS'] = int(ps.min())
data['MIN_NUM_COMPLETIONS_PERIOD'] = ps.idxmin().replace('-','--')
data['MAX_NUM_COMPLETIONS'] = int(ps.max())
data['MAX_NUM_COMPLETIONS_PERIOD'] = ps.idxmax().replace('-','--')

ps=df.groupby('model').id_human.nunique().sort_values()
data['MIN_NUM_COMPLETIONS_MODEL'] = int(ps.min())
data['MIN_NUM_COMPLETIONS_MODEL_NAME'] = rename_model(ps.idxmin())
data['MAX_NUM_COMPLETIONS_MODEL'] = int(ps.max())
data['MAX_NUM_COMPLETIONS_MODEL_NAME'] = rename_model(ps.idxmax())
data['MEDIAN_NUM_COMPLETIONS_MODEL'] = int(ps.median())

df['model_period'] = df.model.map(rename_model) + ' in ' + df.period
ps = df.groupby(['model_period']).id_human.nunique().sort_values()
data['MIN_NUM_COMPLETIONS_MODEL_PERIOD'] = int(ps.min())
data['MIN_NUM_COMPLETIONS_MODEL_PERIOD_NAME'] = ps.idxmin().replace('-','--')
data['MAX_NUM_COMPLETIONS_MODEL_PERIOD'] = int(ps.max())
data['MAX_NUM_COMPLETIONS_MODEL_PERIOD_NAME'] = ps.idxmax().replace('-','--')
data['MEDIAN_NUM_COMPLETIONS_MODEL_PERIOD'] = int(ps.median())

In [ ]:
df = get_rhyme_for_genai_human_completions()

In [ ]:
df_c20l = df[df['period']=='1950-2000']

In [ ]:
df_c20l['model2'] = df_c20l.model.apply(rename_model)
df_c20l = df_c20l[df_c20l.model2!='']
df_c20l['model2'].value_counts()

In [ ]:
pd.set_option('display.max_rows', None)
df_stat = compute_stat_signif(df_c20l, 'model2', 'rhyme_pred_perc').set_index('comparison')
df_stat = df_stat.loc[[x for x in df_stat.index if 'Historical' in x]]
df_stat

In [ ]:
data['CLAUDE_COMPLETION_C20L_EFFECT_SIZE'] = float(round(df_stat.loc['Claude vs (Historical)']['effect_size'], 2))
data['LLAMA_COMPLETION_C20L_EFFECT_SIZE'] = float(round(df_stat.loc['Llama vs (Historical)']['effect_size'], 2))
data['OLMO_COMPLETION_C20L_EFFECT_SIZE'] = float(round(df_stat.loc['Olmo vs (Historical)']['effect_size'], 2))
data['CHATGPT_COMPLETION_C20L_EFFECT_SIZE'] = float(round(df_stat.loc['ChatGPT vs (Historical)']['effect_size'], 2))
data['DEEPSEEK_COMPLETION_C20L_EFFECT_SIZE'] = float(round(df_stat.loc['DeepSeek vs (Historical)']['effect_size'], 2))
data

In [ ]:
df=get_rhyme_for_sample(get_all_memorization_data(), with_sample=True)
data['NUM_WIMBDB'] = len(df[df.found_source_corpus=='open|chadwyck'])
# compute_stat_signif(df, 'found_source_corpus')


In [ ]:
df_rhythm = get_sonnet_rhythm_data(
    sample_by='sonnet_period',
    as_in_paper=True,
    as_replicated=False,
    force=False,
    verbose=True,
)
df = get_rhythm_data_by_syll(df_rhythm)
df

In [ ]:
df

In [ ]:
with open('result_values.json', 'w') as f:
    json.dump(data, f, indent=4)